In [ ]:
import random
import pandas as pd
import math 

In [31]:

def binomial_probability(n, k, p):
    """Calculate binomial probability of getting exactly k successes in n trials"""
    return math.comb(n, k) * (p ** k) * ((1 - p) ** (n - k))

def cumulative_probability_distribution_for_dice_set(n_turns=3, rolled_dice=1, ):
    p = 1 / 6  # Probability of rolling a target value (e.g., "1")
    
    # Initialize the probability distribution for 0 to initial_dice ones
    current_distribution = [0] * (rolled_dice + 1)
    current_distribution[0] = 1.0  


    for turn in range(n_turns):
        next_distribution = [0] * (rolled_dice + 1)
        
        for current_ones in range(rolled_dice + 1):
            remaining_dice = rolled_dice - current_ones
            for new_ones in range(remaining_dice + 1):
                prob_new_ones = binomial_probability(remaining_dice, new_ones, p)
                next_distribution[current_ones + new_ones] += current_distribution[current_ones] * prob_new_ones
        current_distribution = next_distribution
    current_distribution = [round(prob, 4) for prob in current_distribution]
    #for ones, prob in enumerate(current_distribution):
     #   print(f"Probability of getting {ones} ones: {prob}")

    return current_distribution



for a in range(4):
    probability_distribution = cumulative_probability_distribution_for_dice_set(n_turns=a)
    print(probability_distribution)










[1.0, 0]
[0.8333, 0.1667]
[0.6944, 0.3056]
[0.5787, 0.4213]


In [ ]:

class YahtzeeSection:
    def __init__(self, name, is_set, score, number):
        self.name = name
        self.is_set = is_set
        self.score = score
        self.number = number

class Game:
    def __init__(self):
        self.dice_set = [0, 0, 0, 0, 0]
        self.locked_dice = [False, False, False, False, False]
        self.sections = [
            YahtzeeSection("Ones", False, 0, 1),
            YahtzeeSection("Twos", False, 0, 2),
            YahtzeeSection("Threes", False, 0, 3),
            YahtzeeSection("Fours", False, 0, 4),
            YahtzeeSection("Fives", False, 0, 5),
            YahtzeeSection("Sixes", False, 0, 6),
        ]
        self.total_rounds = 6
        self.total_score = 0
        self.total_turns_in_a_round = 3 

    def roll_all_dice(self):
        for i in range(5):
            self.dice_set[i] = random.randint(1, 6)


    def calculate_score(self):
        for section in self.sections:
            self.total_score += section.score

    def play_round(self):
        for turn in range(self.num_turns_in_a_round):
            remaining_turns = self.num_turns_in_a_round - turn
            self.roll_all_dice()
            print("Roll", turn + 1)

            dice_count = []
            for dice_value in range(6):
                
                dice_count.append(self.dice_set.count(dice_value+1))

                simulated_rolled_dice = 5 - dice_count[dice_value]
                distibution =  cumulative_probability_distribution_for_dice_set(n_turns=turn, rolled_dice=(simulated_rolled_dice)) 
                expected_utilities = sum((([distibution] + dice_count[dice_value]) * range(simulated_rolled_dice)) * dice_value)
               
           
            print(expected_utilities)
         



            for die in self.dice:
                print(die.value)
                #for die in dice_to_lock:
                 #   self.lock_die(int(die) - 1)
        #Implement choose optimal section

    def play_game(self):
        for i in range(self.total_rounds):
            self.play_round()
            self.calculate_score()
            print("Total Score:", self.total_score)

            

In [ ]:
game = Game()
game.play_game()

Roll 1
4
5
3
5
4


KeyboardInterrupt: Interrupted by user

: 